# 0. Установка и Импорт

In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install gigachat
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install --upgrade --quiet  yandexcloud
!pip install yandex-chain
!pip install -qU langchain-openai

In [10]:
!pip install pydantic>=1.10.0

In [7]:
# Настраиваем Api ключи
credentials_gigachat = "..."
GOOGLE_API_KEY = '...'
OPENAI_API_KEY = '...'
secret_yandexgpt2 = '...'
folder_id_yandex = "..."

import getpass
import os
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [36]:
from langchain.chat_models.gigachat import GigaChat
from langchain_google_genai import ChatGoogleGenerativeAI
from yandex_chain import ChatYandexGPT
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import re

# 1. Объявляем класс работы с моделями

In [45]:
class Model:
    def __init__(self, model_name):
        match model_name:
            case "gpt-4o":
                self.model = ChatOpenAI(
                    model_name="gpt-4o",
                    openai_api_key=OPENAI_API_KEY
                )
            case 'gigachat':
                self.model = GigaChat(
                    credentials=credentials_gigachat,
                    model='GigaChat',
                    verify_ssl_certs=False
                )
            case 'gemeni':
                self.model = ChatGoogleGenerativeAI(
                    model="gemini-pro",
                    convert_system_message_to_human=True
                )
            case 'yandexGPT':
                self.model = ChatYandexGPT(
                    folder_id=folder_id_yandex,
                    api_key=secret_yandexgpt2
                )

    def get_predict(self, system_prompt, human_prompt):
        message = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=human_prompt),
        ]
        return self.model(message).content

def split_text(text, chunk_size=400):
    """Разделение текста на части с ограничением по количеству слов."""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks


def summarize_block(model, text_block):
    """Создание конспекта блока в формате LaTeX"""
    prompt =  "Преобразуй следующий текст лекции в краткий конспект на русском языке с использованием LaTeX. Используй основные принципы оформления: выделяй важные термины с помощью жирного (\textbf{}) или курсива (\textit{}), добавляй заголовки с командами \section{}, \subsection{}, а также, если нужно, используй списки (\begin{itemize} или \begin{enumerate}). Сохрани чёткость и структурированность текста."
    summary = model.get_predict(prompt, text_block)
    return summary

def generate_latex_document(text, model_name):
    """Генерация полного LaTeX-документа"""
    model = Model(model_name)
    blocks = split_text(text)
    latex_sections = []

    for i, block in enumerate(blocks):
        summary = summarize_block(model, block)
        latex_sections.append(summary)

    latex_document = "\\documentclass{article}\n\\usepackage[english, russian]{babel}\n\\begin{document}\n" + "\n".join(latex_sections) + "\n\\end{document}"
    convert_latex = convert_bold_to_latex(latex_document)
    return convert_latex

def convert_bold_to_latex(text):
    """Заменяет выделение жирным с **текст** на \\textbf{текст} для LaTeX."""
    return re.sub(r"\*\*(.*?)\*\*", r"\\textbf{\1}", text)

# 2. Тестируем!

In [23]:
model = Model("gemeni")
model.get_predict("Напиши привет", 'Привет!')

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Привет!'

In [43]:
with open("/content/лекция.txt", 'r') as f:
    text_to_notes = f.read().strip()

In [47]:
model_name = "gemeni"
latex_output = generate_latex_document(text_to_notes, model_name)

with open("lecture_summary.tex", "w") as file:
    file.write(convert_bold_to_latex(latex_output))

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:362: UserWarning: Convert_system_message_to_hu